In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import accuracy_score,log_loss, r2_score
import pandas as pd
import numpy as np
import os
import itertools

C:\Users\nyaku\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
%matplotlib inline

In [6]:
df = pd.read_csv('dataframes/soco_train_24.csv',header=0)
test_df = pd.read_csv('dataframes/soco_test_A1_24.csv',header=0)
jfiles_df = pd.read_csv('dataframes/jfiles_24.csv',header=0)

In [7]:
jfiles_df

,filename_1_,filename_2_,total_nloc,avg_nloc,avg_ccn,avg_token,function_cnt,ternary,LLC_0,LLC_10,...,multi_comm,doc_comm,sin_space,db_space,tab,avg_var_length,num_type1_var,num_type2_var,total_num_tokens,similarity
0,jfiles/000.java,jfiles/001.java,44.0,28.666667,7.000000,208.166667,4.0,NaN,13.0,12.0,...,NaN,NaN,1065.0,505.0,232.0,2.846066,NaN,6.0,30.0,0.000
1,jfiles/000.java,jfiles/002.java,69.0,34.500000,7.000000,273.000000,0.0,NaN,55.0,1.0,...,NaN,NaN,122.0,15.0,292.0,0.794118,NaN,0.0,371.0,0.143
2,jfiles/000.java,jfiles/003.java,32.0,36.500000,8.000000,262.750000,2.0,NaN,43.0,1.0,...,NaN,NaN,1275.0,682.0,343.0,1.754118,NaN,2.0,219.0,0.000
3,jfiles/000.java,jfiles/004.java,52.0,41.500000,8.750000,303.750000,2.0,NaN,60.0,1.0,...,NaN,NaN,533.0,315.0,358.0,1.987666,NaN,0.0,305.0,0.000
4,jfiles/000.java,jfiles/005.java,221.0,19.111111,4.722222,184.722222,7.0,NaN,111.0,24.0,...,NaN,NaN,3519.0,1554.0,377.0,3.554987,NaN,13.0,470.0,0.000
5,jfiles/000.java,jfiles/006.java,221.0,19.111111,4.722222,184.722222,7.0,NaN,111.0,24.0,...,NaN,NaN,3521.0,1554.0,377.0,3.554987,NaN,13.0,470.0,0.000
6,jfiles/000.java,jfiles/007.java,25.0,41.833333,8.166667,341.000000,4.0,NaN,19.0,2.0,...,NaN,NaN,527.0,283.0,377.0,2.377451,NaN,1.0,323.0,0.000
7,jfiles/000.java,jfiles/008.java,49.0,5.000000,1.500000,15.500000,1.0,NaN,45.0,2.0,...,NaN,NaN,888.0,488.0,372.0,3.270308,4.0,2.0,314.0,0.000
8,jfiles/000.java,jfiles/009.java,44.0,95.000000,9.500000,496.500000,1.0,NaN,9.0,4.0,...,NaN,NaN,2251.0,1073.0,376.0,3.141944,NaN,16.0,119.0,0.000
9,jfiles/000.java,jfiles/010.java,44.0,9.000000,1.500000,72.500000,1.0,NaN,45.0,1.0,...,NaN,NaN,764.0,417.0,375.0,4.683007,8.0,2.0,348.0,0.000


In [ ]:
df.set_index(['filename_1_', 'filename_2_'], inplace=True)
test_df.set_index(['filename_1_', 'filename_2_'], inplace=True)

In [ ]:
df = df.fillna(-1)
test_df = test_df.fillna(-1)

In [ ]:
top_scores_df = df[df['similarity']>0.3]
top_scores_test_df = test_df[test_df['similarity']>0.3]

In [ ]:
top_scores = top_scores_df.append(top_scores_test_df)
top_list = top_scores.values.tolist()

In [ ]:
def calculate_mean_rows(row_list):
    new_rows = []
    index = 0
    for file1_att, file2_att in itertools.combinations(row_list, 2):
        data = np.array([file1_att, file2_att])
        mean = np.average(data, axis=0)
        new_rows.append(mean.tolist())
        index += 1
        if index > 30000:
            return new_rows
    return new_rows

In [ ]:
x = calculate_mean_rows(top_list)
# y = calculate_mean_rows(x)

In [ ]:
additional_scores = x + top_list

In [ ]:
low_scores = df[df['similarity']<0.3].values.tolist()

In [ ]:
final_score_list = additional_scores + low_scores

In [ ]:
columns = df.columns.tolist()

In [ ]:
final_df = pd.DataFrame(final_score_list, columns=columns)

In [ ]:
jfiles_df = pd.read_csv('dataframes/jfiles_24.csv',header=0)
jfiles_df.drop(['filename_1_', 'filename_2_'], axis=1,inplace=True)
jfiles_df = jfiles_df.fillna(-1)

In [ ]:
final_df = final_df.append(jfiles_df)

In [ ]:
# df.drop(columns=['doc_comm', 'rel_prot', 'num_type1_var', 'rel_pub'], inplace=True)

In [ ]:
train, test = train_test_split(final_df)

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=1500,max_depth=20, max_features = 'sqrt', n_jobs = -1)

In [ ]:
rf_regressor.fit(train.drop(['similarity'], axis=1), train['similarity'].values.ravel())

In [ ]:
pred_rf = rf_regressor.predict(test.drop(['similarity'], axis=1).values)

In [ ]:
true_rf = test['similarity'].values

In [ ]:
r2_score(true_rf,pred_rf)

In [ ]:
rf_regressor.feature_importances_

In [ ]:
feature_importances = pd.DataFrame(rf_regressor.feature_importances_,
                                   index = train.drop(['similarity'], axis=1).columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
feature_importances

In [ ]:
jfiles_test = jfiles_df[jfiles_df['similarity']<1.2]

In [ ]:
pred_jfiles = rf_regressor.predict(jfiles_test.drop(['similarity'], axis=1).values)

In [ ]:
true_jfiles = jfiles_test['similarity'].values

In [ ]:
r2_score(true_jfiles,pred_jfiles)

In [ ]:
jfiles_test["pred"] = pred_jfiles

In [ ]:
import pickle

In [ ]:
pkl_filename = "random_forest_model.pkl" 
with open(pkl_filename, 'wb') as file:  
    pickle.dump(rf_regressor, file)